# 1.1 分词

In [1]:
import jieba
import pandas as pd
import tqdm
from harvesttext import HarvestText

ht = HarvestText()

In [2]:
df = pd.read_excel("uzi_all_label_data.xlsx")

In [3]:
stopwords = []
fi = open("stopwords.txt").readlines()
stopwords = [word.strip() for word in fi]

In [5]:
words = set()
for i in tqdm.tqdm(range(len(df))):
    txt = df["标题+内容摘要"][i]
    txt = ht.clean_text(txt)
    temp = list(jieba.cut(txt, cut_all=False))
    for word in temp:
        if word not in stopwords:
            words.add(word)

  0%|          | 0/6143 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 1.124 seconds.
Prefix dict has been built successfully.
100%|██████████| 6143/6143 [00:13<00:00, 464.74it/s]


In [6]:
word2index = {word:i for i, word in enumerate(list(words), 1)}    

In [7]:
len(word2index)

16295

# 1.2 向量化

In [8]:
import numpy as np

In [8]:
avg_len = 0
max_len = 0
vectors = []
for i in tqdm.tqdm(range(len(df))):
    txt = df["标题+内容摘要"][i]
    txt = ht.clean_text(txt)
    temp = list(jieba.cut(txt, cut_all=False))
    vec = []
    for word in temp:
        if word not in stopwords:
            vec.append(word2index[word])
    vectors.append(vec)
    avg_len += len(vec)
    max_len = max_len if len(vec) < max_len else len(vec)
avg_len = avg

100%|██████████| 1500/1500 [00:03<00:00, 493.65it/s]


In [10]:
max_len = 20

In [11]:
for i, vec in enumerate(vectors):
    if len(vec) > max_len:
        vectors[i] = vec[:max_len]
    else:
        padding = list(np.zeros((max_len-len(vec)), dtype=int))
        vectors[i] = vec + padding

In [12]:
with open("uzi_data.txt", "w") as fo:
    for i, vec in enumerate(vectors):
        label = df["Label"][i]
        vector = ",".join([str(v) for v in vec])
        line = vector+"\t"+str(label)+"\n"
        fo.write(line)